# Goal 

The goal of this project is to build one or more forecasting models to determine the demand for a particular product using the other columns as features. 

However, in order to perform time series analysis, I need to get the columns in an acceptable format. I trimmed the features and exported the new dataframe to `data_formatted.csv` I then did time series forcasting for product 1286 and 620 in other notebooks. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('~/Documents/EECS/EECS_731/HW/EECS731_5/data/Historical_Product_Demand.csv')

In [3]:
data.head(5)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


The `product_code` is the product name encoded. The `warehouse` is the warehouse which it was packaged. The `product_category` is the category that the product is in. 

Immediately, I saw that I would need to break up the date, and remove all the underscores from the product code. 

In [4]:
import datetime
data['Date'] = pd.to_datetime(data['Date'], 
        format = '%Y/%m/%d', 
        errors = 'coerce')

In [5]:
data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
data['day'] = data['Date'].dt.day

data['prod_code'] = data['Product_Code'].str.split('_').str[1]
data['warehouse'] = data['Warehouse'].str.split('_').str[1]
data['category'] = data['Product_Category'].str.split('_').str[1]

In [7]:
data = data.drop(columns=['Product_Code'])
data = data.drop(columns=['Warehouse'])
data = data.drop(columns=['Product_Category'])

In [8]:
data.head(5)

,Date,Order_Demand,year,month,day,prod_code,warehouse,category
0,2012-07-27,100,2012.0,7.0,27.0,0993,J,028
1,2012-01-19,500,2012.0,1.0,19.0,0979,J,028
2,2012-02-03,500,2012.0,2.0,3.0,0979,J,028
3,2012-02-09,500,2012.0,2.0,9.0,0979,J,028
4,2012-03-02,500,2012.0,3.0,2.0,0979,J,028


There is a problem with negative numbers here. It seems the dataframe had them formatted with () around the values to signify they are negative. I removed the () and added a - sign. 

I think that the negative sign is a result of a misinput of information. For every neagive number there is a positive number of equal value for the same date. Therefore, I think I must keep the negative number or else I may get inaccurate results. 

In [9]:
data = data.replace({'\(':'-', '\)':''}, regex=True)

In [10]:
data['warehouse'].value_counts()

J    764447
A    153574
S     88200
C     42354
Name: warehouse, dtype: int64

The columns before were all objects 

In [11]:
data['category'] = data['category'].astype('int64')
data['prod_code'] = data['prod_code'].astype('int64')
data['Order_Demand'].astype(float)

0           100.0
1           500.0
2           500.0
3           500.0
4           500.0
            ...  
1048570    1000.0
1048571       1.0
1048572    2500.0
1048573      50.0
1048574       4.0
Name: Order_Demand, Length: 1048575, dtype: float64

In [12]:
data.to_csv('data_formatted.csv')

In [13]:
data['category'].value_counts()

19    481099
5     101671
1      97787
7      82402
21     52011
6      35577
28     31302
11     23208
15     22954
24     20885
9      19738
26     14803
30     12997
32      9296
22      8657
23      7899
18      5239
3       4189
13      3743
20      3490
31      2280
33      1849
8       1560
12      1147
10       976
29       671
17       615
4        329
2         77
16        37
25        35
14        26
27        26
Name: category, dtype: int64

In [14]:
data['prod_code'].value_counts().head(5)

1359    16936
1295    10575
1378     9770
620      9428
1286     8888
Name: prod_code, dtype: int64

The most popular warehouse is warehouse J. The most popular category is 019.  

Some of the most popular codes are 620 and 1286 so I choose those for analysis. 